In [284]:
# TASK QUESTIONS:
# 3.1.2: Which kernel achieved the best accuracy and F1 score?
#   Depends on shuffling. see code

from sklearn.svm import SVC
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
iris_data = load_iris()


In [285]:
state = 7 # Controls shuffling. Same value gives same splitting results
tr_data, te_data, tr_target, te_target = train_test_split(iris_data.data, iris_data.target, test_size=0.2, train_size=0.8, random_state=state)

In [286]:
kernel_types = [ 'linear', 'poly', 'rbf' ]
models = {}

# lin, poly, rbf
for ktype in kernel_types:
    models[ktype] = SVC(kernel=ktype)
    models[ktype].fit(tr_data, tr_target)
    print("done training ", ktype, "accuracy = ", models[ktype].score(te_data, te_target))

# one-vs-one
models['ovo'] = None

# one-vs-rest
models['ovr-linear'] = None
models['ovr-poly']   = None
models['ovr-rbf']    = None


done training  linear accuracy =  0.9666666666666667
done training  poly accuracy =  0.9
done training  rbf accuracy =  0.8666666666666667


In [287]:
# Confusion matrices

np.set_printoptions(precision=2)
debug_enable_plots = False

def plotConfusionMatrix(kernel):
    if not debug_enable_plots: return
    disp = ConfusionMatrixDisplay.from_estimator(
        models[kernel],
        te_data,
        te_target,
        display_labels=iris_data.target_names,
        cmap = plt.cm.Blues,
        normalize = 'true',
    )
    disp.ax_.set_title("Kernel = " + kernel)

plotConfusionMatrix('linear')
plotConfusionMatrix('poly')
plotConfusionMatrix('rbf')
# one vs one
# one vs rest
plt.show()

In [298]:
# F1 + accuracy scores for kernels
best_f1 = (0, None)
best_acc = (0, None)
for k in kernel_types:
    preds = models[k].predict(te_data)
    f1 = f1_score(te_target, preds, average='micro')
    acc = accuracy_score(te_target, preds)
    if acc > best_acc[0]: best_acc = (acc, k)
    if f1 > best_f1[0]: best_f1 = (f1, k)
    print(k + " f1: " + str(f1) + "\n" + k + " acc: " + str(acc))

print("===================== Best results")
print("f1: " + str(best_f1) + "\nacc: " + str(best_acc))


linear f1: 0.9666666666666667
linear acc: 0.9666666666666667
poly f1: 0.9
poly acc: 0.9
rbf f1: 0.8666666666666667
rbf acc: 0.8666666666666667
===================== Best results
f1: (0.9666666666666667, 'linear')
acc: (0.9666666666666667, 'linear')


In [289]:
# Extract support vectors
#print(models['one-vs-one'].support_vectors_)
#print(models['one-vs-rest'].support_vectors_)

In [292]:
# Decision boundry
print(models['linear'].feature_names_in)

AttributeError: 'SVC' object has no attribute 'feature_names_in'